In [0]:
from __future__ import print_function, division
import tensorflow as tf
import numpy as np


In [0]:
batch_size=5
bp_length=15
total_size=50000
actv_layer=4
echoz=3
num_epoch=100
moves=total_size//batch_size//bp_length

In [0]:
def generate():
  x=np.array(np.random.choice(2,total_size,[0.5,0.5]))
  y=np.roll(x,echoz)
  x=x.reshape(batch_size,-1)
  y=y.reshape(batch_size,-1)
  return x,y

In [0]:
x_place=tf.placeholder(tf.float32,shape=[batch_size,bp_length])
y_place=tf.placeholder(tf.int32,shape=[batch_size,bp_length])
actv_zero=tf.placeholder(tf.float32,shape=[batch_size,actv_layer])

In [0]:
x_list=tf.unstack(x_place,axis=1)
y_list=tf.unstack(y_place,axis=1)

In [0]:
W=tf.Variable(np.random.rand(actv_layer+1,actv_layer),dtype=tf.float32)
b=tf.Variable(np.zeros((1,actv_layer)),dtype=tf.float32)
Wy=tf.Variable(np.random.rand(actv_layer,2),dtype=tf.float32)
by=tf.Variable(np.zeros((1,2)),dtype=tf.float32)

In [0]:
states=[]
a_prev=actv_zero
for x_i in x_list:
  x_i=tf.reshape(x_i,[batch_size,1])
  
  conc_mat=tf.concat([a_prev,x_i],1)
  a_prev=tf.tanh(tf.matmul(conc_mat,W)+b)

  states.append(a_prev)
  

In [109]:
losses=[]
logit_series=[tf.matmul(i,Wy)+by for i in states]
print(logit_series[1])
predictions_series = [tf.nn.softmax(logits) for logits in logit_series]


Tensor("add_226:0", shape=(5, 2), dtype=float32)


In [0]:
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logit_series,y_list)]
total_loss = tf.reduce_mean(losses)
train_step=tf.train.AdagradOptimizer(0.3).minimize(total_loss)

In [122]:
u=[]
v=[]
with tf.Session() as ss:
  ss.run(tf.initialize_all_variables())
  for i in range(100):
    x,y=generate()
    cv=0
    for j in range(moves):
      zcurrent_state = np.zeros((batch_size, actv_layer))
      x_part=x[:,j*15:j*15+15]
      y_part=y[:,j*15:j*15+15]
      if i==99:
        u.append(np.transpose(y_part))
      ztotal_loss,ztrain_step,zpredictions_series = ss.run([total_loss, train_step, predictions_series],
                feed_dict={
                    x_place:x_part,
                    y_place:y_part,
                    actv_zero:zcurrent_state
                })
      uv=np.array(zpredictions_series)
      if(i==99):
        v.append(zpredictions_series)
      cv=cv+ztotal_loss
    print(cv)

    

    

166.9286546483636
97.97316148132086
96.02315930277109
95.61830735951662
95.11878239363432
94.66566024720669
94.7995085567236
94.40588743984699
94.15945760905743
94.17194329947233
94.01839189976454
94.04300516843796
93.89521060138941
93.90674435347319
93.75739069283009
93.86845623701811
93.78618051111698
93.67293950915337
93.54048348218203
93.70239320397377
93.55353082716465
93.5011474788189
93.64015181362629
93.47000878304243
93.4304644241929
93.43192621320486
93.52527999132872
93.42601530253887
93.42790588736534
93.3784706518054
93.51636885106564
93.41502905637026
93.27764940261841
93.18540201336145
93.33367617428303
93.31540787965059
93.3102855682373
93.28011790663004
93.12283287197351
93.32484246045351
93.1877975165844
93.21904353052378
93.26283448934555
93.03754805773497
93.12886679172516
93.08658631145954
93.03435031324625
93.22548373043537
93.2087762504816
93.26793533563614
93.0840920433402
93.07612191140652
93.1217665746808
93.02280638366938
93.15648636221886
93.00194756686687
9

In [0]:
q=np.array(zpredictions_series)

In [0]:
lv=np.argmax(q,axis=2)

In [124]:
q.shape

(15, 5, 2)

In [49]:
q[1]

array([[0.4291376 , 0.5708624 ],
       [0.4291376 , 0.5708624 ],
       [0.33782446, 0.66217554],
       [0.37495008, 0.6250499 ],
       [0.4291376 , 0.5708624 ]], dtype=float32)

In [50]:
lv[1]

array([1, 1, 1, 1, 1])

In [0]:
a1=np.array(u)

In [0]:
a2=np.array(np.argmax(v,axis=3))

In [127]:
a1.shape

(666, 15, 5)

In [0]:
zz=np.sum(abs(np.subtract(a1,a2)))

In [134]:
Failaccuracy=zz/500
print(100-Failaccuracy)

90.078
